In [3]:
import keras
import tensorflow
from skimage import io
import os
import cv2
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing import image
from functools import partial
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [21]:
filenames = os.listdir(r"Check")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

data=[]

In [28]:
df=df.sample(frac=1)
df.head(6)

,filename,category
2473,cat.12222.jpg,0
9332,cat.7147.jpg,0
18658,dog.4290.jpg,1
5802,cat.3970.jpg,0
15360,dog.1321.jpg,1
22298,dog.7567.jpg,1


In [5]:
data_gen=ImageDataGenerator(rescale=1./255,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              validation_split=0.2
                             )
train_data=data_gen.flow_from_directory(r"train",
                                  target_size=(128, 128),
                                  batch_size=32,
                                  class_mode='binary',
                                  subset='training')

test_data=data_gen.flow_from_directory(r"train",
                                  target_size=(128, 128),
                                  batch_size=32,
                                  class_mode='binary',
                                  subset='validation')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
train_data.describe

AttributeError: 'DirectoryIterator' object has no attribute 'describe'

In [6]:
Conv=partial(keras.layers.Conv2D,kernel_size=3,activation='relu')


model=Sequential([
    Conv(filters=32,kernel_size=3,input_shape=(128,128,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv(filters=64),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128,kernel_initializer="he_normal",use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    Dense(10,activation="softmax")
])


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               7372800   
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               5

In [14]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
model.save("Cats_Dogs_Classifier.h5")

In [10]:
checkpoint_cb=keras.callbacks.ModelCheckpoint("Cats_Dogs_Classifier.h5",save_best_only=True)

In [11]:
early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [10]:
history= model.fit_generator(train_data,
                   validation_data=test_data,
                   epochs=10,
                   callbacks=[checkpoint_cb,early_stopping_cb])

C:\Users\samee\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
625/625 [==============================] - 320s 512ms/step - loss: 1.2572 - accuracy: 0.6061 - val_loss: 0.7907 - val_accuracy: 0.5782
Epoch 2/10
625/625 [==============================] - 336s 537ms/step - loss: 0.5415 - accuracy: 0.7323 - val_loss: 0.6354 - val_accuracy: 0.6718
Epoch 3/10
625/625 [==============================] - 325s 520ms/step - loss: 0.4904 - accuracy: 0.7667 - val_loss: 0.4839 - val_accuracy: 0.7714
Epoch 4/10
625/625 [==============================] - 327s 523ms/step - loss: 0.4725 - accuracy: 0.7764 - val_loss: 0.4328 - val_accuracy: 0.7940
Epoch 5/10
625/625 [==============================] - 335s 537ms/step - loss: 0.4367 - accuracy: 0.7969 - val_loss: 0.5009 - val_accuracy: 0.7678
Epoch 6/10
625/625 [==============================] - 390s 624ms/step - loss: 0.4203 - accuracy: 0.8086 - val_loss: 0.4439 - val_accuracy: 0.7926
Epoch 7/10
625/625 [==============================] - 436s 697ms/step - loss: 0.3990 - accuracy: 0.8192 - val_loss: 0.4503 -

In [12]:
model=keras.models.load_model("Cats_Dogs_Classifier.h5")

In [13]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(10,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

NameError: name 'history' is not defined

In [13]:
model.evaluate(test_data)

157/157 [==============================] - 44s 278ms/step - loss: 0.3813 - accuracy: 0.8280


[0.38130998611450195, 0.828000009059906]

In [14]:
test_file=os.listdir('test/test')
test_df=pd.DataFrame({'file':test_file})

In [19]:
test_generator = data_gen.flow_from_dataframe(test_df,r'test/test',
                                                 x_col='file',
                                                 y_col=None,
                                                 class_mode=None,
                                                 target_size=(128,128),
                                                 batch_size=32,
                                                 shuffle=False)

Found 12500 validated image filenames.


In [20]:
predict=model.predict(test_generator)

In [21]:
sub = np.around(predict).astype(int)

In [1]:
img = image.load_img(os.path.join(r'test/test',test_file[0]))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title(str(predict[0]))
plt.subplot(1, 4, 4)
img = image.load_img(os.path.join(r'test/test',test_file[1]))
plt.imshow(img)
plt.title(str(predict[1]))
plt.show()

NameError: name 'image' is not defined

In [25]:
model.save_weights("Cats_Dogs_Classifier.h5")